## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

## Load the datasets

In [2]:
king_county_df =  pd.read_csv("data/Food_Establishment_Inspection_Data.csv", low_memory=False)

## Split Description column into two columns

In [3]:
cleaned_data = king_county_df
cleaned_data[["Seating", "Risk"]] = cleaned_data["Description"].str.split(" - ", expand=True)
cleaned_data["Risk"] = king_county_df["Risk"].str.replace("Risk II", "Risk Category II")
cleaned_data.drop(columns="Description", inplace=True)

## Split Violation Description column into two columns

In [4]:
cleaned_data[["Violation Code", "Violation Description"]] = cleaned_data["Violation Description"].str.extract(r"(\d{4})\s*(.*)")

In [5]:
cleaned_data["Violation Description"] = cleaned_data["Violation Description"].str.replace("-", " ").str.strip()

## Convert Inspection Date column to datetime

In [6]:
cleaned_data["Inspection Date"] = pd.to_datetime(cleaned_data["Inspection Date"])

## Remove unimportant columns

In [7]:
cleaned_data.drop(columns=["Name", "Program Identifier", "Address", "Zip Code", "Phone", "Violation_Record_ID"], inplace=True)

## Replace Inspection Closed Business column values with 1 & 0

In [8]:
cleaned_data["Inspection Closed Business"].replace([True, False], [1, 0],inplace=True)

## Grade column

In [9]:
def f(row):
    if 0 <= row["Inspection Score"] <= 11.983561:
        row["Grade"] = 1
    elif 11.983561 < row["Inspection Score"] <= 21.982619:
        row["Grade"] = 2
    elif 21.982619 < row["Inspection Score"] <= 32.785348:
        row["Grade"] = 3
    elif 32.785348 < row["Inspection Score"]:
        row["Grade"] = 4

    
    return row


cleaned_data[cleaned_data["Grade"].isna()] = cleaned_data[cleaned_data["Grade"].isna()].apply(f, axis=1)

## City column

In [10]:
cleaned_data['City'] = cleaned_data['City'].str.lower()
cleaned_data['City'].replace('sea tac', 'seatac', inplace=True)
cleaned_data['City'].replace('vashon island', 'vashon', inplace=True)

## Violation columns

In [11]:
violation_df = pd.read_csv("data/violation.csv", delimiter="|", dtype=object)

In [12]:
def row_maping_function(row):
    if row["Violation Code"] is not np.nan and row["Violation Code"] in violation_df["Violation Code"].values:
        temp = violation_df[violation_df["Violation Code"] == row["Violation Code"]].iloc[0, :]
        row["Violation Description"] = temp["Violation Description"]
        row["Violation Points"] = temp["Violation Points"]
        row["Violation Type"] = temp["Violation Type"]
    return row

cleaned_data.iloc[:, 1:] = cleaned_data.iloc[:, 1:].apply(row_maping_function, axis=1)

## Write the cleaned_data data frame to a csv file

In [13]:
cleaned_data.to_csv("data/king_county_cleaned_data.csv", index=False)